In [1]:
from itertools import product
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import seaborn as sns 
import matplotlib.pylab as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.patheffects as path_effects
import json
import pingouin as pg

In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


In [3]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


In [4]:
filter = "all"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    # OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    # column_sums = np.sum(OFFSIDERS, axis=0)
    # SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    # OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    # column_sums = np.sum(OFFSIDERS, axis=0)
    # SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]


In [5]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]

In [21]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    "SVM":[A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    "RF":[C], 
    "BRF":[C]
}

In [7]:
SEs_name = "SIDER"
metrice = "AUPR"

In [8]:
hyperpars = {}
hyperpars["nested_cv"] = {}
hyperpars["cv"] = {}
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

In [22]:
hyperpars["cv"]["SKR"] = {}
hyperpars["cv"]["SKR"]["pathway"] =(100, 0.9, 10, 100)
hyperpars["cv"]["KRR"] = {}
hyperpars["cv"]["KRR"]["pathway"] = (100, 100)
hyperpars["cv"]["VKR"] = {}
hyperpars["cv"]["VKR"]["pathway"] = (100, 100, 15)
hyperpars["cv"]["SVM"] = {}
hyperpars["cv"]["SVM"]["pathway"] = (10, 10)
hyperpars["cv"]["RF"] = {}
hyperpars["cv"]["RF"]["pathway"] = (15,)
hyperpars["cv"]["BRF"] = {}
hyperpars["cv"]["BRF"]["pathway"] = (15,)

In [10]:
import time

In [11]:
method = "SKR"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (100, 0.9, 10, 100) ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.45871664748046503
AUROCperdrug: 0.8856320744088235
AUPR+AUROCperdrug: 1.3443487218892884
AUPR: 0.39886504381546584
AUROC: 0.8583742465579316
AUPR+AUROC: 1.2572392903733975
-----------
run time:  2.7299623489379883


In [23]:
method = "VKR"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The VKR requires hyperparameter lambda, sigma_X, k
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (100, 100, 15) ------
VKR starts:
VKR ends:
-----------
AUPRperdrug: 0.4588762327062619
AUROCperdrug: 0.8844320863388482
AUPR+AUROCperdrug: 1.34330831904511
AUPR: 0.3991979958326599
AUROC: 0.8581389517046799
AUPR+AUROC: 1.2573369475373397
-----------
run time:  6.085549831390381


In [12]:
method = "KRR"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The KRR requires hyperparameter lambda, sigma_X
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (100, 100) ------
KRR starts:
KRR ends:
-----------
AUPRperdrug: 0.4586145494614498
AUROCperdrug: 0.8851348674276223
AUPR+AUROCperdrug: 1.343749416889072
AUPR: 0.39914209124262745
AUROC: 0.8583574574129526
AUPR+AUROC: 1.25749954865558
-----------
run time:  1.923788070678711


In [17]:
method = "SVM"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The SVM requires hyperparameter c, gamma
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (10, 10) ------
SVM starts:
SVM ends:
-----------
AUPRperdrug: 0.4515203938299791
AUROCperdrug: 0.8780329215278591
AUPR+AUROCperdrug: 1.3295533153578383
AUPR: 0.3931186488444532
AUROC: 0.8486138128633915
AUPR+AUROC: 1.2417324617078447
-----------
run time:  30.726051330566406


In [18]:
method = "RF"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The RF requires hyperparameter k
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (15,) ------
RF starts:
RF ends:
-----------
AUPRperdrug: 0.47212634204237136
AUROCperdrug: 0.8466606098670126
AUPR+AUROCperdrug: 1.318786951909384
AUPR: 0.39716534932872694
AUROC: 0.8243061294982998
AUPR+AUROC: 1.2214714788270267
-----------
run time:  3.716900110244751


In [19]:
method = "BRF"
validation = "cv"
hyperparsOut[validation][method] = {}
results[validation][method] = {}
str = "pathway"
print(f"using feature {str}")
hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
t1 = time.time()
results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs["SIDER"], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)
t2 = time.time()
print("run time: ", t2 - t1)

using feature pathway
The BRF requires hyperparameter k
---------- cv start ----------
--- tuning end ---
target size: 180
------ best hyper pars:  (15,) ------
BRF starts:
BRF ends:
-----------
AUPRperdrug: 0.4721516685314481
AUROCperdrug: 0.8466616909223009
AUPR+AUROCperdrug: 1.318813359453749
AUPR: 0.39715953141505655
AUROC: 0.824306452974596
AUPR+AUROC: 1.2214659843896527
-----------
run time:  105.56408190727234
